In [69]:
import dataiku
import dataikuapi
from dataiku import pandasutils as pdu
import pandas as pd
import logging

In [70]:
design_host = dataiku.api_client().host 
print(design_host)

http://127.0.0.1:20001


In [71]:
design_client = dataikuapi.DSSClient(design_host, "FJk7Jffd8TStmT4Zvx9efC6G5mYZusec") 

In [47]:
curr_log_user = design_client.get_own_user().get_settings().get_raw()["login"]

In [55]:
dss_users = design_client.list_users()
user_info = [user for user in dss_users if user['login'] == curr_log_user]
if 'MLOps' in user_info[0]['groups']:
    logging.info("User {} is in the 'MLOps' group".format(curr_log_user))
else:
    logging.error("User {} is not in the 'MLOps' group".format(curr_log_user))
    raise Exception("User {} is not in the 'MLOps' group".format(curr_log_user))
        

ERROR:root:User admin is not in the 'MLOps' group


Exception: User admin is not in the 'MLOps' group

In [72]:
project = design_client.get_project("DSPIPELINEFRAUD")

In [73]:
project_permissions = project.get_permissions().get("permissions")

In [0]:
dss_users = design_client.list_users()

user_list = []
# Grab list of users where they have active web socket sessions
for user in dss_users:
    if user['activeWebSocketSesssions'] != 0:
        user_list.append(user['displayName'])
print(user_list)

In [0]:
dss_users

In [18]:
project_key = project.project_key
print(project_key)

DSPIPELINEFRAUD


In [7]:
#bundle_version_type = config.get("bundle_version_type", "")
bundle_version_type = "minor"

In [8]:
#exported_bundles = ["DSPIPELINEFRAUD_v1", "DSPIPELINEFRAUD_v1.1", "DSPIPELINEFRAUD_v1.2", "DSPIPELINEFRAUD_v2", "DSPIPELINEFRAUD_v2.1"]

In [34]:
#version_id = 1
exported_bundles = project.list_exported_bundles()['bundles']
list_bundles = []
for bundles in exported_bundles:
    list_bundles.append(bundles['bundleId'])
if len(list_bundles) == 0: 
    if bundle_version_type == "major":
        version_id = 1
    elif bundle_version_type == "minor":
        raise Exception("This is the first time you deploy the project, it should be a major version!") 
elif len(list_bundles) > 0:
    last_deployed_bundles = list_bundles[-1]
    last_version = last_deployed_bundles.split('v')[1].split('_')
    if bundle_version_type == "major":
        version_id = int(last_version[0]) + 1
    elif bundle_version_type == "minor":
        if len(last_version) == 1:
            version_id = last_version[0] + '_1'
        elif len(last_version) == 2:
            version_id = last_version[0] + '_' + str(int(last_version[1]) + 1)

bundle_id = project_key + '_v' + str(version_id)

In [35]:
bundle_id

'DSPIPELINEFRAUD_v1_2'

In [25]:
list_bundles = list(set(list_bundles))

In [26]:
list_bundles

['DSPIPELINEFRAUD_v1', 'DSPIPELINEFRAUD_v1_1']

'DSPIPELINEFRAUD_v1_1'

In [37]:
project.list_exported_bundles()

{'bundles': [{'bundleId': 'DSPIPELINEFRAUD_v2',
   'contentSummary': {'allAnalysisData': False,
    'configContent': {'analysis': 2,
     'dashboards': 3,
     'datasets': 24,
     'insights': 16,
     'jupyterNotebooks': 3,
     'managedFolders': 1,
     'modelEvaluationStores': 0,
     'recipes': 16,
     'savedModels': 1,
     'scenarios': 0,
     'sqlNotebooks': 0},
    'includedDatasets': [{'name': 'transactions_2018',
      'size': 5404878,
      'type': 'UploadedFiles',
      'usedTransferMethod': 'UPLOAD'},
     {'name': 'transactions_2017',
      'size': 7423251,
      'type': 'UploadedFiles',
      'usedTransferMethod': 'UPLOAD'},
     {'name': 'merchant_info',
      'size': 3362746,
      'type': 'UploadedFiles',
      'usedTransferMethod': 'UPLOAD'},
     {'name': 'cardholder_info',
      'size': 2543734,
      'type': 'UploadedFiles',
      'usedTransferMethod': 'UPLOAD'}],
    'includedManagedFolders': [],
    'includedModelEvaluationStores': [],
    'includedSavedModels'

In [38]:
host = "http://localhost:15000/"
auto_client = dataikuapi.DSSClient(host, "Da1cTKi0eO3VMUUvCq7TuIfSHRBzY3OA") 
project = auto_client.get_project("DSPIPELINEFRAUD")

In [39]:
project.list_imported_bundles()

{'bundles': [{'bundleId': 'DSPIPELINEFRAUD_v2',
   'contentSummary': {'allAnalysisData': False,
    'configContent': {'analysis': 2,
     'dashboards': 3,
     'datasets': 24,
     'insights': 16,
     'jupyterNotebooks': 3,
     'managedFolders': 1,
     'modelEvaluationStores': 0,
     'recipes': 16,
     'savedModels': 1,
     'scenarios': 0,
     'sqlNotebooks': 0},
    'includedDatasets': [{'name': 'transactions_2018',
      'size': 5404878,
      'type': 'UploadedFiles',
      'usedTransferMethod': 'UPLOAD'},
     {'name': 'transactions_2017',
      'size': 7423251,
      'type': 'UploadedFiles',
      'usedTransferMethod': 'UPLOAD'},
     {'name': 'merchant_info',
      'size': 3362746,
      'type': 'UploadedFiles',
      'usedTransferMethod': 'UPLOAD'},
     {'name': 'cardholder_info',
      'size': 2543734,
      'type': 'UploadedFiles',
      'usedTransferMethod': 'UPLOAD'}],
    'includedManagedFolders': [],
    'includedModelEvaluationStores': [],
    'includedSavedModels'

In [74]:
writers = [permission["group"] for permission in project_permissions if permission.get("writeProjectContent", False)==True]

In [75]:
writers

['MLOps', 'cc_fraud_project']

In [56]:
project_deployer_client = dataikuapi.DSSClient('http://localhost:20000/', "FJk7Jffd8TStmT4Zvx9efC6G5mYZusec") 

In [57]:
project_owner = project.get_permissions().get("owner")
published_project = project_deployer_client.get_project("DSPIPELINEFRAUD") 

In [59]:
published_project

In [60]:
published_project_settings = published_project.get_settings()
raw_permissions = published_project_settings.get_raw()

In [62]:
published_project_settings

In [63]:
logging.info("Project owner will again be {}".format(project_owner))

In [64]:
# Set the new project owner on the Deployer
raw_permissions['owner'] = project_owner

In [79]:
raw_permissions['permissions']

KeyError: 'permissions'

In [77]:
for writer in writers:
    if writer == 'MLOps':
        permission = {
                'admin': True, 
                'deploy': True, 
                'group': writer, 
                'read': True, 
                'write': True
        }
        raw_permissions['permissions'].append(permission)

KeyError: 'permissions'

In [0]:
        
    # this part deduplicates any permissions given to the same group.
permissions = { each['group'] : each for each in raw_permissions['permissions'] }.values()
list_permissions = list(permissions)
raw_permissions['permissions'] = list_permissions
published_project_settings.save()